### Copyright (C) Infineon Technologies AG 2025
 
Copyright (c) 2025, Infineon Technologies AG, or an affiliate of Infineon Technologies AG. All rights reserved.
This software, associated documentation and materials ("Software") is owned by Infineon Technologies AG or one of its affiliates ("Infineon") and is protected by and subject to worldwide patent protection, worldwide copyright laws, and international treaty provisions. Therefore, you may use this Software only as provided in the license agreement accompanying the software package from which you obtained this Software. If no license agreement applies, then any use, reproduction, modification, translation, or compilation of this Software is prohibited without the express written permission of Infineon.

Disclaimer: UNLESS OTHERWISE EXPRESSLY AGREED WITH INFINEON, THIS SOFTWARE IS PROVIDED AS-IS, WITH NO WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ALL WARRANTIES OF NON-INFRINGEMENT OF THIRD-PARTY RIGHTS AND IMPLIED WARRANTIES SUCH AS WARRANTIES OF FITNESS FOR A SPECIFIC USE/PURPOSE OR MERCHANTABILITY. Infineon reserves the right to make changes to the Software without notice. You are responsible for properly designing, programming, and testing the functionality and safety of your intended application of the Software, as well as complying with any legal requirements related to its use. Infineon does not guarantee that the Software will be free from intrusion, data theft or loss, or other breaches ("Security Breaches"), and Infineon shall have no liability arising out of any Security Breaches. Unless otherwise explicitly approved by Infineon, the Software may not be used in any application where a failure of the Product or any consequences of the use thereof can reasonably be expected to result in personal injury.

# Anomaly Detection with MLP Autoencoder

This notebook demonstrates how to build and train a Multi-Layer Perceptron (MLP) based autoencoder for anomaly detection using the Controlled Anomalies Time Series (CATS) dataset and the helper functions from `modelling_helper.py`.

## Introduction

Anomaly detection is a critical capability for monitoring complex systems and identifying unusual patterns that may indicate equipment failures, security breaches, or operational issues. In industrial applications, effective anomaly detection provides:

- **Predictive maintenance** - Early detection of equipment degradation before catastrophic failure
- **Quality assurance** - Identification of process deviations that affect product quality
- **Security monitoring** - Detection of unusual patterns that may indicate cyberattacks or system intrusions
- **Operational optimization** - Identification of inefficient or suboptimal system behaviors
- **Cost reduction** - Prevention of costly downtime through early anomaly identification

## Autoencoder Approach

This notebook uses an **autoencoder neural network** approach for anomaly detection:

- **Training Phase** - The autoencoder learns to reconstruct normal system behavior from multivariate time series data
- **Detection Phase** - Anomalies are identified by measuring reconstruction error - high reconstruction error indicates anomalous behavior
- **Unsupervised Learning** - The model learns normal patterns without requiring labeled anomalous examples during training

## Notebook Structure

This notebook walks through the complete anomaly detection workflow:

1. **Data Loading & Exploration** - Load the CATS dataset, understand its structure, and visualize time series patterns
2. **Data Preprocessing** - Normalize features, create time series sequences, and prepare training/validation splits
3. **Model Definition** - Build an MLP-based autoencoder architecture optimized for time series reconstruction
4. **Model Training** - Train the autoencoder on normal data with early stopping and learning rate scheduling
5. **Model Evaluation** - Assess reconstruction performance and validate anomaly detection capability
6. **Model Export** - Convert the trained model to ONNX format for deployment
7. **Hardware Deployment** - Compile the model for AURIX&trade; microcontroller deployment
8. **Performance Analysis** - Evaluate execution timing and resource usage on target hardware

The methodologies demonstrated are applicable to various industrial monitoring scenarios including manufacturing processes, power systems, transportation networks, and IoT sensor networks.

## Import Libraries and Helper Functions

In [ ]:
import sys
import os

# Add parent directory to path for importing central scripts
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import project modules
import CentralScripts.helper_functions as cs
import modelling_helper as mh

# Load and Visualize Data

## Dataset: Controlled Anomalies Time Series (CATS)

This notebook uses the **Controlled Anomalies Time Series (CATS) Dataset** for anomaly detection model training. The CATS dataset is a synthetic multivariate time series dataset specifically designed for benchmarking anomaly detection algorithms.

**Dataset Source:** [https://zenodo.org/records/8338435](https://zenodo.org/records/8338435)

### Key Dataset Characteristics:

- **Multivariate:** 17 variables including:
  - 4 Control commands/actuations
  - 3 Environmental stimuli/external forces  
  - 10 Telemetry sensor readings (position, temperature, pressure, etc.)

- **Scale:** 5 million timestamps at 1Hz sampling frequency
  - First 1 million: Nominal (normal) behavior for learning baseline
  - Last 4 million: Mixed nominal and anomalous segments for evaluation

- **Anomalies:** 200 precisely labeled anomalous segments with different types and known ground truth
- **Root Cause Analysis:** Metadata includes root cause channels and affected channels for each anomaly
- **Clean Signals:** Noise-free data allows controlled robustness testing

The dataset simulates a complex dynamical system and is particularly suitable for:
- Semi-supervised anomaly detection (novelty detection)
- Unsupervised anomaly detection (outlier detection)  
- Root cause analysis and explainability evaluation
- Algorithm robustness testing

### Citation

```
Patrick Fleith. (2023). Controlled Anomalies Time Series (CATS) Dataset (Version 2) 
[Data set]. Solenix Engineering GmbH. https://doi.org/10.5281/zenodo.8338435
```

In [ ]:
# load data
train, test = mh.load_data()

## Data Overview

In [ ]:
mh.plot_time_steps(
    df=train,
    n_steps=1000,
)

# Prepare input sequences for model training

- `sequence_length` defines the number of time steps takes as input for the model. Adjust this to see its effect on detection quality. Larger `seqence_length` requires a larger input layer of the auto encoder and therefore results in overall larger models requiring more memory.
- From the train data random sequences of length `sequence_length` are selected. A higher number ensures better coverage of the normal behaviour of the train data but increases training time.
- Input data is proved as a tensor of sequences with shape `n_samples * sequence_length * n_features`

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled, scaler = mh.normalize_data(df=train)
test_scaled, scaler = mh.normalize_data(df=test, scaler=scaler)

# seelct the length of sequences
sequence_length = 30

sequences = mh.sample_sequences(
    df=train_scaled,
    sequence_length=sequence_length,
    n_samples=10000,  # Use a smaller number for quicker testing
)

# Split sequences: 95% train, 5% validation
sequences_train, sequences_val = train_test_split(
    sequences, test_size=0.05, random_state=42
)

print(
    f"Train sequences: {len(sequences_train)}, Validation sequences: {len(sequences_val)}"
)

# Build an MLP based autoencoder

- Set the number of layers: it will be the same number on both encoder and decoder sides. A single layer proved to get good results
- The bottleneck defines the compression of the data. Play with this number to see what works best.
- As loss the Mean Squared Error (MSE) is used. Model performance later is assessed using this metric.

In [ ]:
model = mh.mlp_autoencoder(
    input_dim=sequences_train.shape[1] * sequences_train.shape[2],
    bottleneck=64,
    layers=1,
    p_drop=0.1,
)

# Train the model

- The training process uses early stopping to avoid overfitting.
- To ensure proper convergence learning rate reduction is used when training stalls.

In [ ]:
X_train = sequences_train.reshape(sequences_train.shape[0], -1)
X_val = sequences_val.reshape(sequences_val.shape[0], -1)

history = model.fit(
    x=X_train,
    y=X_train,
    epochs=10,
    batch_size=64,
    validation_data=[X_val, X_val],
    callbacks=mh.get_callbacks(),
)
cs.plot_training_history(history)

# Model evaluation

- Make model predictions on the train & test data sets and calculate the reconstruction error as Mean Squared Error
- Compare the MSE distributions for train and test data -> both distributions should strongly overlap!
- Define an anomaly threshold `thresh` for the MSE above which to consider a sequence as anomalous. Typically, this is set to the 95th or above percentile of the MSE of the train or test set. Adjusting this threshold shifts the balance between false positive and false negative detections.
- Check model functionality on anomaly data

In [ ]:
import numpy as np
import pandas as pd

y_train = model.predict(X_train)
err_train = np.sum((y_train - X_train) ** 2, axis=1)

res_train = pd.DataFrame({"MSE": err_train, "Train_test": "Train", "Label": "clean"})

test_scaled, scaler = mh.normalize_data(df=test, scaler=scaler)

sequences_test = mh.sample_sequences(
    df=test_scaled,
    sequence_length=sequence_length,
    n_samples=5000,
)

X_test = sequences_test.reshape(sequences_test.shape[0], -1)

y_test = model.predict(X_test)
err_test = np.sum((y_test - X_test) ** 2, axis=1)

res_test = pd.DataFrame({"MSE": err_test, "Train_test": "Test", "Label": "clean"})

res = pd.concat([res_train, res_test], ignore_index=True)


quantile = 0.95  # 95th percentile, adjust as needed
threshold = res_train["MSE"].quantile(quantile)

print(f"Set anomaly threshold at: {threshold:.2f}")

## Plot reconstruction error distributions

- Mean squared error (MSE) as metric as used for model training loss.
- MSE for anomaly free data from train and test data.

In [ ]:
mh.plot_reconstruction_error(res, threshold)

## Check detection quality

- Plot an anomalous time series.
- The meta data data frame contains the information for 200 labelled anomalies from the test data set. Select one and see the detection quality.
- Calculate the MSE at each time point

In [ ]:
meta_data = pd.read_csv("data/metadata.csv")
meta_data.head()

In [ ]:
import re

# selecting an anomaly from the meta data (0 - 199)
ii = 11

start = meta_data["start_time"][ii]
start = pd.to_datetime(start)
end = meta_data["end_time"][ii]
end = end = pd.to_datetime(end)

root_cause = meta_data["root_cause"][ii]
affected = meta_data["affected"][ii]
affected = re.findall(r"'([^']*)'", affected)[0]

print(f"Anomaly from {start} to {end}, root cause: {root_cause}, affected: {affected}")

# Load the anomalous data directly from data.csv
anomaly_rows = mh.load_anomaly_data_from_csv("data/data.csv", start, end)

# Set timestamp as index for easier manipulation
anomaly_rows = anomaly_rows.set_index("timestamp")

# Normalize the anomaly data using the same scaler
anomaly_rows_scaled = mh.normalize_data(df=anomaly_rows, scaler=scaler)[0]

# Update anomaly_data dictionary with the new data
anomaly_data = {
    "rows": anomaly_rows,
    "rows_scaled": anomaly_rows_scaled,
    "start": start,
    "end": end,
    "root_cause": root_cause,
    "affected": affected,
}

print(f"Loaded anomaly data shape: {anomaly_rows.shape}")
print(f"Time range: {anomaly_rows.index.min()} to {anomaly_rows.index.max()}")

## Create sequences of anomalous data for model predictions and calculate reconstruction error

- Sequences as rolling window over the selected anomalous time series.

In [ ]:
seqs = mh.sequential_sequences(
    df=anomaly_rows_scaled,
    sequence_length=sequence_length,
)

y_pred = model.predict(seqs.reshape(seqs.shape[0], -1))
err = np.sum((y_pred - seqs.reshape(seqs.shape[0], -1)) ** 2, axis=1)

err = pd.DataFrame({"MSE": err, "timestamp": anomaly_rows.index[sequence_length - 1 :]})

anomaly_data["err"] = err

## Plot results

- At each time point plot the reconstruction error and compare it against the selected threshold to label anomalous time points

In [ ]:
mh.plot_anomaly_detection(anomaly_data, threshold)

# Export Model to ONNX

- Generate a model name based on its architecture and loss
- Save the model together with a test in- & output

In [ ]:
test_loss = model.evaluate(X_test, X_test, verbose=0)

model_name = mh.generate_model_name(model, test_loss)

origin = "tf"

input_target = sequences_train[0].reshape(-1).astype(np.float32)
output_target = cs.get_predictions(origin, model, input_target)

cs.save_all(model_name, input_target, output_target, model, origin)

# Convert Model for AURIX&trade; TC3x/TC4x deployment

- Make sure the docker container with conversion tools is running
- Submit the model together with test data to the container and download the generated code, binary and log files.
- Results are saved in the `out/<model_name>/test_<model_name>/<target>/` folder

In [ ]:
cs.ensure_docker_container()

In [ ]:
from CentralScripts.python_flask_client import CallTools

model_folder, onnx_model_file = cs.get_output_paths(model_name)

for target in ["TC3", "TC4"]:
    tool = CallTools(
        folder=model_folder, url="http://localhost:8080/convert", target=target
    )
    tool.convert_model()

## Simulated instruction counts

- For each node in the neural network the number of instruction counts is extracted from the log file and plotted.
- You can inspect which node is a computational bottleneck and adjust your network.

In [ ]:
cs.plot_instruction_counts(model_name)